In [1]:
import pandas as pd
from datetime import datetime
import xlrd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

China: datos faltantes antes de 02/01/2006

In [2]:
cotizaciones = pd.read_excel('../data/cotizaciones 1997.xlsx', sheet_name=0)
cotizaciones_usd = pd.read_excel('../data/cotizaciones 1997.xlsx', sheet_name=1)
lista_paises = cotizaciones_usd.columns[1:-3]

var_cotizaciones_usd = cotizaciones_usd.copy()
var_cotizaciones_usd=var_cotizaciones_usd.groupby(['anio','mes'],as_index=False).mean()
var_cotizaciones_usd['Período']=pd.to_datetime('1/'+var_cotizaciones_usd.mes.astype(str)+'/'+var_cotizaciones_usd.anio.astype(str), format='%d/%m/%Y')
# var_cotizaciones_usd=var_cotizaciones_usd.fillna(method='bfill')
for i in range(len(lista_paises)):
    var_cotizaciones_usd[lista_paises[i]
                         ] = var_cotizaciones_usd[lista_paises[i]].pct_change(12)

var_cotizaciones_usd.drop(['mes','anio'],axis=1,inplace=True)
cotizaciones_usd.drop(['dia','mes','anio'],axis=1,inplace=True)

var_cotizaciones_usd

,Brasil,Canadá,Chile,Argentina,México,Uruguay,China,India,Japón,Reino Unido,Suiza,Zona Euro,Vietnam,dia,Período
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.5,1997-11-01
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,1997-12-01
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,1998-01-01
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.5,1998-02-01
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,1998-03-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,0.038633,0.033117,0.263720,0.334644,0.027530,-0.062577,0.039471,0.067078,0.237190,0.150238,0.055457,0.160289,0.015889,16.0,2022-07-01
297,-0.021361,0.025148,0.158636,0.391966,0.001418,-0.064497,0.049537,0.073300,0.232497,0.152667,0.046792,0.162669,0.024046,16.0,2022-08-01
298,-0.008130,0.052017,0.178988,0.456643,0.003015,-0.041754,0.085520,0.090031,0.298236,0.214553,0.055411,0.189250,0.037399,15.5,2022-09-01
299,-0.049925,0.101106,0.172687,0.532530,-0.021607,-0.057261,0.119263,0.098541,0.301186,0.211029,0.078307,0.178437,0.068010,16.0,2022-10-01


In [3]:
def plot_tasa_deva(df=var_cotizaciones_usd, anio='2007', mes='01', dia='01'):
    idx = df[df.Período == f'{str(anio)}-{str(mes)}-{str(dia)}'].index[0]
    var_cotizaciones_usd = df[df.index >= idx].reset_index(drop=True)

    x = var_cotizaciones_usd["Período"]
    # Datos
    cotizaciones_plot = go.Figure()
    cotizaciones_plot.add_trace(go.Scatter(
        name="USD-Real", x=x, y=var_cotizaciones_usd["Brasil"], mode="lines"))
    cotizaciones_plot.add_trace(go.Scatter(
        name="USD-Yuan", x=x, y=var_cotizaciones_usd["China"], mode="lines"))
    # cotizaciones_plot.add_trace(go.Scatter(name = "ARS-USD", x=x, y=var_cotizaciones_usd["Argentina"], mode = "lines"))
    cotizaciones_plot.add_trace(go.Scatter(
        name="USD-Euro", x=x, y=var_cotizaciones_usd["Zona Euro"], mode="lines"))

    # Propiedades
    cotizaciones_plot.update_xaxes(showgrid=True)
    # cotizaciones_plot.update_yaxes(title_text="Tasa de devaluación", tickformat= ',.0%')
    cotizaciones_plot.update_yaxes(
        title_text="Tasa de devaluación", tickformat=',.0%')
    cotizaciones_plot.update_layout(separators=",.", font_family="Georgia",
                                    height=600, width=1000,
                                    template='none',
                                    title_text=f"Tasa de devaluación de las monedas de los tres principales socios comerciales <br><sup>Variación interanual con frecuencia mensual",
                                    legend=dict(yanchor="top", y=1.05, xanchor="left", x=0.36, orientation='h'))

    note = 'Fuente: BCRA'
    cotizaciones_plot.add_annotation(showarrow=False, text=note, font=dict(size=12), xref='paper', x=0.1, yref='paper', y=-0.1,
                                     xanchor='right', yanchor='auto', xshift=0, yshift=0,)

    # Marca de agua
    cotizaciones_plot.add_annotation(showarrow=False, text='@MartinBasualdo0', font=dict(size=16), font_family="arial", opacity=0.4,
                                     xref='paper', yref='paper', x=0.5, y=0.5)
    return cotizaciones_plot


def evolucion_monedas(df=cotizaciones_usd, anio='2006', mes='01', dia='01'):
    idx = df[df.Período == f'{str(anio)}-{str(mes)}-{str(dia)}'].index[0]
    cotizaciones_usd = df[df.index >= idx].reset_index(drop=True)
    x = cotizaciones_usd["Período"]

    # Datos
    cotizaciones_plot = go.Figure()
    cotizaciones_plot.add_trace(go.Scatter(name=f"Brasil", x=x, y=cotizaciones_usd["Brasil"], mode="lines"))
    cotizaciones_plot.add_trace(go.Scatter(name=f"China", x=x, y=cotizaciones_usd["China"], mode="lines"))
    cotizaciones_plot.add_trace(go.Scatter(name=f"Euro", x=x, y=cotizaciones_usd["Zona Euro"], mode="lines"))

    # Propiedades
    cotizaciones_plot.update_xaxes(showgrid=True)
    # cotizaciones_plot.update_yaxes(title_text="Tasa de devaluación")
    cotizaciones_plot.update_layout(separators=",.", font_family="Georgia",
                                    height=600, width=1000,
                                    template='none',
                                    title_text=f"Evolución de las paridades con el USD de los tres principales socios comerciales <br><sup>Frecuencia diaria",
                                    legend=dict(yanchor="top", y=1.05, xanchor="left", x=0.36, orientation='h'))

    note = 'Fuente: BCRA'
    cotizaciones_plot.add_annotation(showarrow=False, text=note, font=dict(size=12), xref='paper', x=0.1, yref='paper', y=-0.1,
                                     xanchor='right', yanchor='auto', xshift=0, yshift=0,)

    # Marca de agua
    cotizaciones_plot.add_annotation(showarrow=False, text='@MartinBasualdo0', font=dict(size=16), font_family="arial", opacity=0.4,
                                     xref='paper', yref='paper', x=0.5, y=0.5)
    return cotizaciones_plot


def indice_monedas(df=cotizaciones_usd, anio='2006', mes='01', dia='01'):
    idx = df[df.Período == f'{str(anio)}-{str(mes)}-{str(dia)}'].index[0]
    df = df[df.index >= idx].reset_index(drop=True)
    lista_paises=df.columns[1:]
    var_cotizaciones_usd = df.copy()
    base = var_cotizaciones_usd[var_cotizaciones_usd.Período == '2019-01-01'].index[0]
    for i in range(len(lista_paises)):
        var_cotizaciones_usd[lista_paises[i]] = var_cotizaciones_usd[lista_paises[i]
                                                                     ]/var_cotizaciones_usd[lista_paises[i]].iloc[base]*100
    # var_cotizaciones_usd.dropna(inplace=True)

    x = var_cotizaciones_usd["Período"]

    # Datos
    cotizaciones_plot = go.Figure()
    cotizaciones_plot.add_trace(go.Scatter(
        name="USD-Real", x=x, y=var_cotizaciones_usd["Brasil"], mode="lines"))
    cotizaciones_plot.add_trace(go.Scatter(
        name="USD-Yuan", x=x, y=var_cotizaciones_usd["China"], mode="lines"))
    # cotizaciones_plot.add_trace(go.Scatter(name = "ARS-USD", x=x, y=var_cotizaciones_usd["Argentina"], mode = "lines"))
    cotizaciones_plot.add_trace(go.Scatter(
        name="USD-Euro", x=x, y=var_cotizaciones_usd["Zona Euro"], mode="lines"))

    # Propiedades
    cotizaciones_plot.update_xaxes(showgrid=True)
    # cotizaciones_plot.update_yaxes(title_text="Tasa de devaluación", tickformat= ',.0%')
    cotizaciones_plot.update_yaxes(
        title_text="Base enero 2019=100")
    cotizaciones_plot.update_layout(separators=",.", font_family="Georgia",
                                    margin ={'b': 50,'l':50,'r':15},
                                    height=600, width=900,
                                    template='none',
                                    title_text=f"Evolución de las monedas de los tres principales socios comerciales <br><sup>índice simple con base enero 2019=100",
                                    legend=dict(yanchor="top", y=1.05, xanchor="left", x=0.3, orientation='h'))

    note = 'Fuente: BCRA'
    cotizaciones_plot.add_annotation(showarrow=False, text=note, font=dict(size=12), xref='paper', x=0.1, yref='paper', y=-0.09,
                                     xanchor='right', yanchor='auto', xshift=0, yshift=0,)

    # Marca de agua
    cotizaciones_plot.add_annotation(showarrow=False, text='@MartinBasualdo0', font=dict(size=16), font_family="arial", opacity=0.4,
                                     xref='paper', yref='paper', x=0.5, y=0.5)
    return cotizaciones_plot


# plot_tasa_deva(anio=1999)

evolucion_monedas(anio=2021,mes=1)

# indice_monedas(anio=2007)


In [4]:
cotizaciones_usd=cotizaciones_usd.fillna(method='bfill')
cotizaciones=cotizaciones.fillna(method='bfill')

In [5]:
itcrm = pd.read_excel('../data/ITCRMSerie.xlsx', header=1, skipfooter=4)

ponderadores = pd.read_excel('../data/ITCRMSerie.xlsx', sheet_name=2, header=1)
ponderadores.Período = pd.to_datetime(
    ponderadores['Período'], format='%d/%m/%Y')


def evolucion_ponderaciones():
    x = ponderadores["Período"]

    # Datos
    cotizaciones_plot = go.Figure()
    for index, pais in enumerate(ponderadores.columns[1:]):
        cotizaciones_plot.add_trace(go.Scatter(
            name=f"{pais}", x=x, y=ponderadores[pais], mode="lines"))

    # Propiedades
    cotizaciones_plot.update_xaxes(showgrid=True)
    cotizaciones_plot.update_yaxes(title_text="Ponderaciones %")
    cotizaciones_plot.update_layout(separators=",.", font_family="Georgia",
                                    height=600, width=1000,
                                    template='none',
                                    title_text=f"Evolución de las ponderaciones publicadas por el BCRA <br><sup>Frecuencia mensual",
                                    legend=dict(yanchor="top", y=-.05, xanchor="left", x=0.36, orientation='h'))

    note = 'Fuente: BCRA'
    cotizaciones_plot.add_annotation(showarrow=False, text=note, font=dict(size=12), xref='paper', x=0.1, yref='paper', y=-0.1,
                                     xanchor='right', yanchor='auto', xshift=0, yshift=0,)

    # Marca de agua
    cotizaciones_plot.add_annotation(showarrow=False, text='@MartinBasualdo0', font=dict(size=16), font_family="arial", opacity=0.4,
                                     xref='paper', yref='paper', x=0.5, y=0.5)
    return cotizaciones_plot


evolucion_ponderaciones()


In [6]:
# Tipo de cambio real

fin = itcrm.Período.iloc[-1].strftime('%m/%d/%Y')
inicio = itcrm.Período[0].strftime('%m/%d/%Y')
ponderadores.Período = ponderadores.Período.apply(
    lambda x: x.strftime('%d/%m/%Y'))
itcrm.Período = itcrm.Período.apply(lambda x: x.strftime('%d/%m/%Y'))

monthDates = pd.DataFrame({
    'Período': pd.date_range(start=inicio, end=fin, freq='d').strftime('%d/%m/%Y')
})

ponderaciones_completo = (monthDates[['Período']].merge(ponderadores, on='Período', how='left').fillna(method='ffill')
                          .merge(itcrm[['Período']], on='Período', how='right'))

# ponderaciones_completo[ponderaciones_completo.Período=='01/01/2004']
ponderaciones_completo


,Período,Brasil,Canadá,Chile,Estados Unidos,México,Uruguay,China,India,Japón,Reino Unido,Suiza,Zona Euro,Vietnam
0,01/01/1997,29.562711,0.797194,5.098276,20.522159,2.301729,3.338386,2.323775,0.854147,2.321107,2.448984,0.795496,29.620689,0.015347
1,02/01/1997,29.562711,0.797194,5.098276,20.522159,2.301729,3.338386,2.323775,0.854147,2.321107,2.448984,0.795496,29.620689,0.015347
2,03/01/1997,29.562711,0.797194,5.098276,20.522159,2.301729,3.338386,2.323775,0.854147,2.321107,2.448984,0.795496,29.620689,0.015347
3,04/01/1997,29.562711,0.797194,5.098276,20.522159,2.301729,3.338386,2.323775,0.854147,2.321107,2.448984,0.795496,29.620689,0.015347
4,05/01/1997,29.562711,0.797194,5.098276,20.522159,2.301729,3.338386,2.323775,0.854147,2.321107,2.448984,0.795496,29.620689,0.015347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9448,14/11/2022,25.942405,1.094186,4.422352,12.695719,2.462603,2.300716,19.623359,6.658939,1.572981,1.088982,1.752113,17.539508,2.846135
9449,15/11/2022,25.942405,1.094186,4.422352,12.695719,2.462603,2.300716,19.623359,6.658939,1.572981,1.088982,1.752113,17.539508,2.846135
9450,16/11/2022,25.942405,1.094186,4.422352,12.695719,2.462603,2.300716,19.623359,6.658939,1.572981,1.088982,1.752113,17.539508,2.846135
9451,17/11/2022,25.942405,1.094186,4.422352,12.695719,2.462603,2.300716,19.623359,6.658939,1.572981,1.088982,1.752113,17.539508,2.846135


In [7]:
cotizaciones_final = (ponderaciones_completo[['Período']].merge(cotizaciones, on='Período', how='left')
                     .fillna(method='ffill')
                     .drop_duplicates('Período', ignore_index=True)
                     )

indice = cotizaciones_final[cotizaciones_final.Período ==
                           cotizaciones.Período.iloc[0]].index
cotizaciones_final = cotizaciones_final[cotizaciones_final.index >= indice[0]].reset_index(
    drop=True)

cotizaciones_final


,Período,Brasil,Canadá,Chile,Estados Unidos,México,Uruguay,China,India,Japón,Reino Unido,Suiza,Zona Euro,Vietnam
0,21/11/1997,0.904404,0.706700,0.002366,1.00,0.121803,0.100604,0.375548,0.139803,0.007959,1.691000,0.706400,1.181000,0.001639
1,22/11/1997,0.904404,0.706700,0.002366,1.00,0.121803,0.100604,0.375548,0.139803,0.007959,1.691000,0.706400,1.181000,0.001639
2,23/11/1997,0.904404,0.706700,0.002366,1.00,0.121803,0.100604,0.375548,0.139803,0.007959,1.691000,0.706400,1.181000,0.001639
3,24/11/1997,0.902039,0.704700,0.002332,1.00,0.121507,0.101214,0.375548,0.139803,0.007888,1.692500,0.713100,1.181000,0.001639
4,25/11/1997,0.902039,0.706200,0.002332,1.00,0.121507,0.101214,0.375548,0.139803,0.007851,1.685500,0.711800,1.181000,0.001639
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9124,14/11/2022,30.457292,121.657392,0.181132,161.78,8.360940,4.051499,22.885840,1.990832,1.153923,190.253280,171.686299,167.215808,0.006524
9125,15/11/2022,30.439354,122.115095,0.182928,162.12,8.366404,4.074903,23.010432,1.998952,1.163819,192.809316,171.700911,168.377832,0.006535
9126,16/11/2022,30.317789,121.956163,0.178309,162.47,8.420316,4.082982,22.888901,1.998246,1.165662,193.225571,172.126285,168.692601,0.006554
9127,17/11/2022,29.844343,122.106369,0.176593,162.78,8.371605,4.082738,22.741625,1.993692,1.160311,192.682686,170.825900,168.461022,0.006560


In [8]:
ponderaciones_final = cotizaciones_final[['Período']].merge(
    ponderaciones_completo, on='Período')
itcrm_final = cotizaciones_final[['Período']].merge(itcrm, on='Período')

# Canasta_t=itcr_t/cotiz_t

canastas = pd.DataFrame()
canastas['Período'] = itcrm_final.Período
lista_paises = ponderaciones_final.columns[1:]

for i in range(len(lista_paises)):
    canastas[lista_paises[i]] = itcrm_final[itcrm_final.columns[i+2]] / \
        cotizaciones_final[cotizaciones_final.columns[i+1]]
canastas


,Período,Brasil,Canadá,Chile,Estados Unidos,México,Uruguay,China,India,Japón,Reino Unido,Suiza,Zona Euro,Vietnam
0,21/11/1997,130.059660,104.781714,40770.936999,71.409668,662.833523,753.634398,155.766407,392.283357,3735.757239,49.662325,95.871446,67.303052,35926.152314
1,22/11/1997,130.075399,104.784613,40775.552011,71.417219,663.122113,753.892907,155.730046,392.337812,3736.300047,49.667075,95.876932,67.310472,35932.909805
2,23/11/1997,130.091140,104.787513,40780.167545,71.424771,663.410828,754.151505,155.693694,392.392273,3736.842934,49.671825,95.882419,67.317892,35939.668568
3,24/11/1997,130.553885,104.791961,41398.736328,71.432324,661.396446,749.863486,155.659231,388.943674,3743.499339,49.714873,96.055695,67.808972,35848.346308
4,25/11/1997,130.569684,104.794861,41403.422403,71.439877,661.684410,750.120702,155.621016,386.738559,3744.043272,49.719628,96.061192,67.491937,35906.268941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9124,14/11/2022,2.830797,0.857306,489.867088,0.635815,12.236028,26.191414,3.766931,47.154087,84.408493,0.428787,0.531016,0.566947,14085.890999
9125,15/11/2022,2.826017,0.855648,488.967870,0.634689,12.216152,26.141167,3.758912,47.070966,84.254001,0.428033,0.529957,0.565895,14059.191219
9126,16/11/2022,2.821245,0.853994,488.070303,0.633565,12.196309,26.091015,3.750911,46.987991,84.099791,0.427280,0.528901,0.564846,14032.542048
9127,17/11/2022,2.816482,0.852342,487.174383,0.632443,12.176498,26.040960,3.742926,46.905163,83.945864,0.426528,0.527847,0.563798,14005.943391


In [9]:
indice = itcrm[itcrm.Período == cotizaciones.Período.iloc[0]].index-1
itcrm_rezago = itcrm[(itcrm.index >= indice[0]) & (
    itcrm.index <= itcrm.index[-2])].reset_index(drop=True)

itcrm_rezago


,Período,ITCRM,ITCRB Brasil,ITCRB Canadá,ITCRB Chile,ITCRB Estados Unidos,ITCRB México,ITCRB Uruguay,ITCRB China,ITCRB India,ITCRB Japón,ITCRB Reino Unido,ITCRB Suiza,ITCRB Zona Euro,ITCRB Vietnam,ITCRB Sudamérica*
0,20/11/1997,83.657504,117.792696,73.854316,96.633511,71.401256,80.278131,75.678549,58.510714,55.259287,29.474352,84.278556,68.481639,80.226803,59.036581,96.701585
1,21/11/1997,83.421457,117.626477,74.049237,96.464037,71.409668,80.735112,75.818635,58.497763,54.842390,29.732892,83.978991,67.723590,79.484905,58.883718,96.636383
2,22/11/1997,83.431203,117.640711,74.051286,96.474956,71.417219,80.770263,75.844642,58.484107,54.850003,29.737212,83.987024,67.727465,79.493667,58.894794,96.653436
3,23/11/1997,83.440949,117.654947,74.053335,96.485876,71.424771,80.805429,75.870658,58.470455,54.857617,29.741533,83.995057,67.731341,79.502430,58.905872,96.670494
4,24/11/1997,83.610626,117.764696,73.846895,96.541853,71.432324,80.364298,75.896683,58.457513,54.375492,29.528723,84.142423,68.497316,80.082395,58.756192,96.734411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9124,13/11/2022,89.989199,85.017819,104.068182,88.522271,102.356824,100.856660,105.341188,85.474359,93.936912,97.962234,81.459298,90.750023,94.408435,91.370265,92.960426
9125,14/11/2022,90.661857,86.218413,104.297630,88.730605,102.862201,102.304696,106.114488,86.209380,93.875865,97.400902,81.578092,91.168133,94.802459,91.895127,93.687836
9126,15/11/2022,90.822544,86.022139,104.487568,89.445915,102.895796,102.205265,106.522718,86.494195,94.092601,98.056407,82.528665,90.994152,95.284236,91.872948,93.996924
9127,16/11/2022,90.449636,85.533925,104.149779,87.027328,102.935284,102.696774,106.529146,85.854221,93.893565,98.031931,82.561357,91.037754,95.285331,91.967204,93.030133


In [10]:
cotizaciones_final[cotizaciones_final.columns[0+1]]

0        0.904404
1        0.904404
2        0.904404
3        0.902039
4        0.902039
          ...    
9124    30.457292
9125    30.439354
9126    30.317789
9127    29.844343
9128    30.547755
Name: Brasil, Length: 9129, dtype: float64

### Productoria

Primero calculo la original, que parte del paso para calcular el itcrm

In [11]:
productoria_original = pd.DataFrame()
productoria_original['Período'] = itcrm_final.Período

for i in range(len(lista_paises)):
    productoria_original[lista_paises[i]] = ((cotizaciones_final[cotizaciones_final.columns[i+1]]
                                    # / cotizaciones_final["Estados Unidos"]
                                     * canastas[canastas.columns[i+1]]
                                     / itcrm_rezago[itcrm_rezago.columns[i+2]]
                                     )**(ponderaciones_final[ponderaciones_final.columns[i+1]]/100))

productoria_original['productoria'] = productoria_original[productoria_original.columns[1:]
                                         ].product(axis=1)

productoria_original

,Período,Brasil,Canadá,Chile,Estados Unidos,México,Uruguay,China,India,Japón,Reino Unido,Suiza,Zona Euro,Vietnam,productoria
0,21/11/1997,0.999555,1.000026,0.999918,1.000025,1.000112,1.000062,0.999994,0.999933,1.000246,0.999907,0.999932,0.997506,0.999999,0.997217
1,22/11/1997,1.000038,1.000000,1.000005,1.000022,1.000009,1.000011,0.999994,1.000001,1.000004,1.000002,1.000000,1.000030,1.000000,1.000117
2,23/11/1997,1.000038,1.000000,1.000005,1.000022,1.000009,1.000011,0.999994,1.000001,1.000004,1.000002,1.000000,1.000030,1.000000,1.000117
3,24/11/1997,1.000294,0.999972,1.000027,1.000022,0.999892,1.000011,0.999994,0.999922,0.999797,1.000046,1.000069,1.001955,0.999999,1.002001
4,25/11/1997,1.000038,1.000022,1.000005,1.000022,1.000009,1.000011,0.999993,0.999950,0.999871,0.999895,0.999989,0.998741,1.000000,0.998547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9124,14/11/2022,1.003645,1.000024,1.000104,1.000625,1.000351,1.000168,1.001682,0.999957,0.999910,1.000016,1.000081,1.000731,1.000163,1.007475
9125,15/11/2022,0.999409,1.000020,1.000355,1.000041,0.999976,1.000088,1.000647,1.000154,1.000106,1.000126,0.999967,1.000889,0.999993,1.001772
9126,16/11/2022,0.998525,0.999965,0.998788,1.000049,1.000118,1.000001,0.998544,0.999859,0.999996,1.000004,1.000008,1.000002,1.000029,0.995894
9127,17/11/2022,0.995489,0.999992,0.999491,1.000017,0.999817,0.999954,0.998317,0.999731,0.999899,0.999950,0.999832,0.999434,0.999972,0.991915


In [12]:
productoria_equilibrio = pd.DataFrame()
productoria_equilibrio['Período'] = itcrm_final.Período

for i in range(len(lista_paises)):
    productoria_equilibrio[lista_paises[i]] = ((cotizaciones_final[cotizaciones_final.columns[i+1]]
                                    / cotizaciones_final["Estados Unidos"]
                                     * canastas[canastas.columns[i+1]]
                                     / itcrm_rezago[itcrm_rezago.columns[i+2]])
                                    **(ponderaciones_final[ponderaciones_final.columns[i+1]]/100))

productoria_equilibrio['productoria'] = productoria_equilibrio[productoria_equilibrio.columns[1:]
                                         ].product(axis=1)

productoria_equilibrio

,Período,Brasil,Canadá,Chile,Estados Unidos,México,Uruguay,China,India,Japón,Reino Unido,Suiza,Zona Euro,Vietnam,productoria
0,21/11/1997,0.999555,1.000026,0.999918,1.000025,1.000112,1.000062,0.999994,0.999933,1.000246,0.999907,0.999932,0.997506,0.999999,0.997217
1,22/11/1997,1.000038,1.000000,1.000005,1.000022,1.000009,1.000011,0.999994,1.000001,1.000004,1.000002,1.000000,1.000030,1.000000,1.000117
2,23/11/1997,1.000038,1.000000,1.000005,1.000022,1.000009,1.000011,0.999994,1.000001,1.000004,1.000002,1.000000,1.000030,1.000000,1.000117
3,24/11/1997,1.000294,0.999972,1.000027,1.000022,0.999892,1.000011,0.999994,0.999922,0.999797,1.000046,1.000069,1.001955,0.999999,1.002001
4,25/11/1997,1.000038,1.000022,1.000005,1.000022,1.000009,1.000011,0.999993,0.999950,0.999871,0.999895,0.999989,0.998741,1.000000,0.998547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9124,14/11/2022,0.268245,0.945890,0.798654,0.524605,0.882583,0.889717,0.369202,0.712673,0.923028,0.946133,0.914813,0.410093,0.865370,0.006227
9125,15/11/2022,0.266968,0.945864,0.798781,0.524159,0.882206,0.889603,0.368669,0.712714,0.923178,0.946216,0.914675,0.410007,0.865171,0.006179
9126,16/11/2022,0.266582,0.945790,0.797453,0.524020,0.882285,0.889482,0.367739,0.712401,0.923046,0.946078,0.914678,0.409489,0.865149,0.006130
9127,17/11/2022,0.265640,0.945796,0.797947,0.523876,0.881978,0.889401,0.367517,0.712219,0.922928,0.946007,0.914487,0.409119,0.865053,0.006094


In [13]:
# Tipo de cambio de equilibrio = 1/(productoria_t*ITCRM_rezagado/promedio)
prom_inicio = itcrm[itcrm.Período == '01/07/2002'].index[0]
prom_final = itcrm[itcrm.Período == '01/01/2007'].index[0]
promedio_2002_2007 = itcrm[(itcrm.index >= prom_inicio) & (
    itcrm.index < prom_final)].mean()[0]

tc_equilibrio = pd.DataFrame()
tc_equilibrio['Período'] = itcrm_final.Período

tc_equilibrio['tc_equilibrio'] = 1 / \
    (productoria_equilibrio.productoria*itcrm_rezago['ITCRM ']/promedio_2002_2007)
tc_equilibrio['tc_oficial_mayorista'] = cotizaciones_final["Estados Unidos"]
tc_equilibrio['Período'] = pd.to_datetime(
    tc_equilibrio['Período'], format='%d/%m/%Y')
tc_equilibrio['brecha'] = tc_equilibrio.tc_equilibrio / \
    tc_equilibrio.tc_oficial_mayorista-1

tc_equilibrio


C:\Users\mbasualdo\AppData\Local\Temp\ipykernel_14804\999697203.py:5: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



,Período,tc_equilibrio,tc_oficial_mayorista,brecha
0,1997-11-21,1.928727,1.00,0.928727
1,1997-11-22,1.928575,1.00,0.928575
2,1997-11-23,1.928350,1.00,0.928350
3,1997-11-24,1.924499,1.00,0.924499
4,1997-11-25,1.927237,1.00,0.927237
...,...,...,...,...
9124,2022-11-14,287.121348,161.78,0.774764
9125,2022-11-15,287.215713,162.12,0.771624
9126,2022-11-16,289.022476,162.47,0.778928
9127,2022-11-17,291.934218,162.78,0.793428


In [14]:
idx = tc_equilibrio[tc_equilibrio.Período == '2006-01-01'].index[0]
tc_equilibrio[tc_equilibrio.index >= idx]

,Período,tc_equilibrio,tc_oficial_mayorista,brecha
2963,2006-01-01,3.003639,3.032,-0.009354
2964,2006-01-02,3.004377,3.030,-0.008456
2965,2006-01-03,2.999979,3.038,-0.012515
2966,2006-01-04,2.969478,3.039,-0.022877
2967,2006-01-05,2.972846,3.050,-0.025296
...,...,...,...,...
9124,2022-11-14,287.121348,161.780,0.774764
9125,2022-11-15,287.215713,162.120,0.771624
9126,2022-11-16,289.022476,162.470,0.778928
9127,2022-11-17,291.934218,162.780,0.793428


In [15]:
def plot_brecha(df=tc_equilibrio, anio_ini='2003', mes='01', dia='01'):
    idx = df[df.Período == f'{str(anio_ini)}-{str(mes)}-{str(dia)}'].index[0]
    df = df[df.index >= idx].reset_index(drop=True)
    x = df["Período"]
    ultima_fecha = df.Período.iloc[-1]
    ultimo_tc_equil = round(df.tc_equilibrio.iloc[-1], 2)
    ultimo_tc_may = df.tc_oficial_mayorista.iloc[-1]
    ultima_brecha = df.brecha.iloc[-1]

    # Datos
    cotizaciones_plot = make_subplots(specs=[[{"secondary_y": True}]])
    cotizaciones_plot.add_trace(go.Scatter(
        name="Tipo de cambio de equilibro", x=x, y=df["tc_equilibrio"], mode="lines"), secondary_y=False)
    cotizaciones_plot.add_trace(go.Scatter(name="Tipo de cambio oficial mayorista",
                                x=x, y=df["tc_oficial_mayorista"], mode="lines"), secondary_y=False)
    cotizaciones_plot.add_trace(go.Scatter(
        name="Brecha (der)", x=x, y=df["brecha"], mode="lines"), secondary_y=True)

    # Propiedades
    cotizaciones_plot.update_xaxes(showgrid=True)
    cotizaciones_plot.update_yaxes(
        title_text="Cotizaciones", secondary_y=False, zeroline=False)
    cotizaciones_plot.update_yaxes(
        title_text="Brecha (devaluación requerida)", tickformat=',.0%', secondary_y=True)
    cotizaciones_plot.update_layout(separators=",.", font_family="Georgia",
                                    margin ={'b': 50,'l':50,'r':15},
                                    height=600, width=900,
                                    template='none',
                                    title_text=f"Tipo de cambio nominal de equilibrio macroeconómico {str(anio_ini)}-{str(ultima_fecha.year)}<br><sup>Tipo de cambio que dejaría al ITCRM igual al promedio jul/02-dic/06: {str(round(promedio_2002_2007,1)).replace('.',',')}",
                                    title_font=dict(size=20),
                                    legend=dict(yanchor="top", y=1.05, xanchor="left", x=0.15, orientation='h'))

    # Flechas
    cotizaciones_plot.add_annotation(x=ultima_fecha, y=ultimo_tc_equil, align="left",
                                     text='$'+str(ultimo_tc_equil).replace('.', ','), showarrow=True, arrowhead=1)
    cotizaciones_plot.add_annotation(ax=0, ay=35, x=ultima_fecha, y=ultimo_tc_may, align="left",
                                     text='$'+str(ultimo_tc_may).replace('.', ','), showarrow=True, arrowhead=1)
    
    #Regímnes cambiarios
    anio_ini=int(anio_ini)
    if anio_ini<2007: cotizaciones_plot.add_vline(x="2006-12-12", line_width=1, line_dash="dash", line_color="Black",opacity=0.5)
    if anio_ini<2012: cotizaciones_plot.add_vline(x="2011-10-28", line_width=1, line_dash="dash", line_color="Black",opacity=0.5)
    if anio_ini<2016:cotizaciones_plot.add_vline(x="2015-12-17", line_width=1, line_dash="dash", line_color="Black",opacity=0.5)
    if anio_ini<2019:cotizaciones_plot.add_vline(x="2018-09-26", line_width=1, line_dash="dash", line_color="Black",opacity=0.5)
    if anio_ini<2020:cotizaciones_plot.add_vline(x="2019-09-01", line_width=1, line_dash="dash", line_color="Black",opacity=0.5)
    if anio_ini<2004:cotizaciones_plot.add_annotation(showarrow=False, text=f'TC competitivo y estable<br>"Crawling peg"',
        font=dict(size=13), font_family="georgia",x='2004-12-1', y=75)
    if anio_ini<2009:cotizaciones_plot.add_annotation(showarrow=False, text=f'Apreciación cambiaria<br>Inicio intervención INDEC',
        font=dict(size=13), font_family="georgia",x='2009-7-1', y=200)
    if anio_ini<2013:cotizaciones_plot.add_annotation(showarrow=False, text=f'CEPO cambiario<br>INDEC intervenido',
        font=dict(size=13), font_family="georgia",x='2013-11-1', y=250)
    if anio_ini<2017:cotizaciones_plot.add_annotation(showarrow=False, text=f'TC flotante<br>"Flotación sucia"',
        font=dict(size=13), font_family="georgia",x='2017-5-1', y=75)
    if anio_ini<2019:cotizaciones_plot.add_annotation(showarrow=False, text=f"Bandas cambiarias",
        font=dict(size=13), font_family="georgia",x='2019-03-1', y=175,textangle=90)
    if anio_ini<2021:cotizaciones_plot.add_annotation(showarrow=False, text=f'Apreciación cambiaria<br>Con CEPO',
        font=dict(size=13), font_family="georgia",x='2021-9-1', y=50) 
    
    # Nota al pie
    note = 'Fuente: BCRA'
    cotizaciones_plot.add_annotation(showarrow=False, text=note, font=dict(size=12), xref='paper', x=0.1, yref='paper', y=-0.1,
                                     xanchor='right', yanchor='auto', xshift=0, yshift=0,)

    # Marca de agua
    cotizaciones_plot.add_annotation(showarrow=False, text='@MartinBasualdo0', font=dict(size=16), font_family="arial", opacity=0.4,
                                     xref='paper', yref='paper', x=0.5, y=0.5)

    cotizaciones_plot.add_annotation(showarrow=False, text=f"Devaluación requerida al {ultima_fecha.strftime('%d/%m/%y')}:<br>{'{:.1%}'.format(ultima_brecha).replace('.',',')}", font=dict(size=14), font_family="georgia",
                                     # xref='paper', yref='paper',
                                     x='2019-05-30', y=250)
    return cotizaciones_plot


plot_brecha(anio_ini=2003)


In [16]:
writer = pd.ExcelWriter(f'../output/ITCRM historico.xlsx', engine='xlsxwriter')
itcrm.to_excel(writer, sheet_name='ITCRM', index=False)
ponderaciones_completo.to_excel(
    writer, sheet_name='ponderaciones', index=False)
cotizaciones_final.to_excel(writer, sheet_name='cotizaciones', index=False)
canastas.to_excel(writer, sheet_name='canastas', index=False)
tc_equilibrio.to_excel(writer, sheet_name='tc_equilibrio', index=False)
writer.save()


In [17]:
plot_brecha(anio_ini=2003).write_html('../output/grafico_brecha.html')
indice_monedas(anio=2007).write_html('../output/indice_monedas.html')
plot_tasa_deva(anio=2007).write_html('../output/grafico_devaluacion_socios.html')